In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import bioio_ome_zarr
from pathlib import Path
import os
import zarr
import tifffile
import gc  # for freeing memory
from skimage import exposure
from skimage.transform import resize
from fuse_n5_xml_to_tif import pix_size

In [ ]:
from pathlib import Path 
from bioio import BioImage


In [ ]:
# ----- File Paths -----
# base_path per OS
# macOS
# base_path = Path("/Users/thomasnegrello/mnt/tnegrell_ista/siegegrp/ThNe/development_retina/raw_images_n5/")
# Ubuntu
# base_path = Path("/mnt/gdrive/ThNe/development_retina/raw_images_n5/")
# Windows
base_path = Path("Z:/siegegrp/ThNe/development_retina/")
#debian
# base_path = Path("/run/user/1009/gvfs/smb-share:server=fs.ista.ac.at,share=drives/tnegrell/group/siegegrp/ThNe/development_retina/")

#
fuse_folder = Path("trila_fuse_mg/P10/F/L/")
file_name = "microglia_Age_P10_Sex_F_Side_L_Animal_1.zarr"
# fuse_folder = Path("trila_fuse_mg/P21/F/L/")
# file_name = "microglia_Age_P21_Sex_F_Side_L_Animal_1"

file_path = base_path / fuse_folder / file_name

zarr_path = file_path.with_suffix(".zarr")


In [ ]:
img = BioImage(zarr_path, reader=bioio_ome_zarr.Reader)

In [ ]:
z = zarr.open(zarr_path, mode='r')
z.tree()

In [ ]:
arr = z['0'] 
z_max_index = int(0.65*arr.shape[2])
mg_img = arr[0,0,:z_max_index]

In [ ]:
in_memory_size_gb = arr.nbytes / (1024 ** 3)
print(f"In-memory size: {in_memory_size_gb:.2f} GB")


In [ ]:
arr

In [ ]:
import zarr

# Check the chunking scheme
z = zarr.open(zarr_path, mode='r')
print("Chunking scheme:", z['0'].chunks)  # This will give you the chunk sizes

# Now, make sure your slices don't cross the chunk boundaries


In [ ]:
import zarr

# Open the Zarr group
z = zarr.open(zarr_path, mode='r')

# List all the keys (arrays/datasets) in the group
print("Contents of the Zarr group:", list(z.keys()))

# Access the specific array within the group (replace '0' with the actual key)
arr = z['0']  # or use the appropriate key if it's different

# Now, you can access the shape and chunking information of the array
print("Shape of the array:", arr.shape)
print("Chunking scheme of the array:", arr.chunks)


In [ ]:
import numpy as np
import tifffile
import os

# Open the Zarr array
arr = zarr.open(zarr_path, mode='r')['0']
# Zarr chunking
C, T, Z, Y, X = arr.shape
chunk_size_z, chunk_size_y, chunk_size_x = arr.chunks[2], arr.chunks[3], arr.chunks[4]  # Get chunk sizes in Z, Y, X

# Crop Z axis
z_end = int(0.65 * Z)

# Ensure z_end is aligned with chunk boundaries in Z (multiple of 64)
z_end_aligned = (z_end // chunk_size_z) * chunk_size_z

# Define XY chunking with overlap
n_chunks_x = 2
n_chunks_y = 2
overlap = 600

x_chunk_size = X // n_chunks_x
y_chunk_size = Y // n_chunks_y

output_dir = base_path / fuse_folder
os.makedirs(output_dir, exist_ok=True)

chunk_id = 0
for i in range(n_chunks_y):
    for j in range(n_chunks_x):
        y_start = max(0, i * y_chunk_size - (overlap if i > 0 else 0))
        y_end = min(Y, (i + 1) * y_chunk_size + (overlap if i < n_chunks_y - 1 else 0))
        
        x_start = max(0, j * x_chunk_size - (overlap if j > 0 else 0))
        x_end = min(X, (j + 1) * x_chunk_size + (overlap if j < n_chunks_x - 1 else 0))
        
        # Slice lazily from Zarr (aligned with chunks)
        chunk = arr[:, :, 0:z_end_aligned, y_start:y_end, x_start:x_end]
        chunk_np = chunk[:]  # Load into memory
        print('done', chunk_id)
        # Optional: remove singleton dimensions if C=1 and T=1
        if chunk_np.shape[0] == 1 and chunk_np.shape[1] == 1:
            chunk_np = chunk_np[0, 0]  # Now shape (Z, Y, X)

        # Save as TIFF
        output_path = os.path.join(output_dir, f"chunk_{chunk_id}.tiff")
        tifffile.imwrite(output_path, chunk_np, bigtiff=True)
        print(f"Saved chunk {chunk_id} to {output_path}")
        chunk_id += 1
